In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib
from skimage import io
import time

%matplotlib inline

In [ ]:
# self-written scripts
import sys
sys.path.insert(0, 'Python_Scripts')

import util
import surf_handling
import surf_hog_analysis

In [ ]:
df = pd.read_csv('data/train_complete.csv')

In [ ]:
df_defectless = df.query('Defect == 0')

In [ ]:
df_defectless

In [ ]:
def add_keypoints_to_frame(df, surf_object):
    # prepare dictionary to gather data
    surf_images = {'keypoints': [],
                   'ImageId': [],
                   'NumberKP': []
                  }

    print('processing images...')
    start = time.time()

    for idx, image_id in enumerate(df.ImageId):
        surf_images['ImageId'].append(image_id)

        # `image` so far holds just the path to the image. Convert to image file
        image = io.imread("data/train_images/" + image_id)
        # Find keypoints and descriptors directly
        kp, des = surf_object.detectAndCompute(image, None)

        surf_images['keypoints'].append(kp)
        surf_images['NumberKP'].append(len(kp))
        if idx % 500 == 0 and idx != 0:
            print(f'image number {idx} processed...')

    end = time.time()
    print('processing done.')
    print('required time:', end - start)
    
    temp = pd.DataFrame.from_dict(surf_images)
    
    return df.merge(temp, on='ImageId')

In [ ]:
surf = cv2.xfeatures2d.SURF_create(400)

In [ ]:
df_defectless_with_kp = add_keypoints_to_frame(df_defectless, surf)

In [ ]:
defectless_complete = util.add_blackness_attributes(df_defectless_with_kp, 'train_images')

In [ ]:
defectless_complete

In [ ]:
defectless_complete.query('PercentageBlack <= 0.5 and NumberKP < 51')

In [ ]:
num_kp_list_sorted = df_defectless_with_kp.sort_values(by='NumberKP', ascending=True).NumberKP.to_numpy()
num_kp_list = df_defectless_with_kp.NumberKP.to_numpy()

In [ ]:
num_kp_list_sorted[:1000]

In [ ]:
#np.where(num_kp_list < 51)

In [ ]:
defectless_kp_less_51 = df_defectless_with_kp.query('NumberKP < 51')
defectless_kp_less_51

In [ ]:
surf_hog_analysis.print_batch(defectless_kp_less_51, defectless_kp_less_51.ClassId, show_keypoints=True)

In [ ]:
defectless_kp_less_51.to_csv('data/defectless_less_51_kp.csv', sep=',', index=False)